#  RNN/LSTM model for generating fake Python code

Setting up a basic framework using TensorFlow's Keras API to create a Sequential model with LSTM, Dense, and Embedding layers. This kind of setup is commonly used for sequence-based modeling tasks like text generation, sentiment analysis, or time series prediction.

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import LSTM, Dense, Embedding
from tensorflow.keras.models import Sequential
import numpy as np

A list of Python code snippets as strings

In [ ]:
python_code = [
    "def add(a, b): return a + b",
    "def subtract(a, b): return a - b",
    "def multiply(a, b): return a * b",
    "x = 5\ny = 10\nresult = x + y",
    "my_list = [1, 2, 3, 4, 5]\nfiltered_list = [num for num in my_list if num % 2 == 0]",
    "text = 'Hello, World!'\nreversed_text = text[::-1]",
    "num = 10\nfactorial = 1\nfor i in range(1, num + 1):\n    factorial *= i",
    "students = {'Alice': 20, 'Bob': 22, 'Charlie': 21}\nages = [age for age in students.values()]",
    "class Dog:\n    def __init__(self, name, breed):\n        self.name = name\n        self.breed = breed\n    def bark(self):\n        return 'Woof!'",
    "def factorial(n):\n    if n == 0 or n == 1:\n        return 1\n    else:\n        return n * factorial(n-1)",
    "def fibonacci(n):\n    a, b = 0, 1\n    result = []\n    while a < n:\n        result.append(a)\n        a, b = b, a + b\n    return result",
    "def is_prime(n):\n    if n <= 1:\n        return False\n    for i in range(2, int(n**0.5) + 1):\n        if n % i == 0:\n            return False\n    return True",
    "def greet(name):\n    print('Hello, ' + name + '! Welcome to the world of Python.')",
    "def check_palindrome(s):\n    return s == s[::-1]",
    "def calculate_area_circle(radius):\n    return 3.1416 * radius ** 2",
    "result = 5 * 3 + 2",
    "my_tuple = (1, 2, 3, 4, 5)\ndouble_values = [2 * value for value in my_tuple]",
    "text = 'Python is fun!'\nupper_text = text.upper()",
    "numbers = [1, 2, 3, 4, 5]\nproduct = 1\nfor num in numbers:\n    product *= num",
    "my_dict = {'a': 1, 'b': 2, 'c': 3}\nkeys = [key for key in my_dict.keys()]",
    "class Rectangle:\n    def __init__(self, width, height):\n        self.width = width\n        self.height = height\n    def calculate_area(self):\n        return self.width * self.height",
]

 Utilizing TensorFlow's Keras Tokenizer to preprocess a list of Python code snippets. The Tokenizer is being used to convert the text data (Python code in this case) into numerical sequences suitable for training neural networks.

In [ ]:
tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n', lower=True)
tokenizer.fit_on_texts(python_code)
total_words = len(tokenizer.word_index) + 1

 Generating sequences of tokens from the preprocessed Python code snippets using the previously created tokenizer. These sequences are created in a way that each sequence includes an increasing number of tokens from the original code snippets.

In [ ]:
input_sequences = []
for line in python_code:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

max_sequence_len = max([len(seq) for seq in input_sequences])
input_sequences = np.array(tf.keras.preprocessing.sequence.pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))


 Preparing the input sequences (X) and their corresponding labels (y) to be used for training a neural network model.

In [ ]:
# Create input sequences and labels
X = input_sequences[:, :-1]
y = tf.keras.utils.to_categorical(input_sequences[:, -1], num_classes=total_words)

Defines an LSTM-based neural network model using TensorFlow's Keras API.

In [ ]:
# Define the LSTM model
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
model.add(LSTM(150))
model.add(Dense(total_words, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

 Training the defined neural network model using the input sequences (X) and their corresponding labels (y).

In [ ]:
model.fit(X, y, epochs=100, verbose=1)

Epoch 1/100
9/9 [==============================] - 2s 25ms/step - loss: 4.5344 - accuracy: 0.0270
Epoch 2/100
9/9 [==============================] - 0s 26ms/step - loss: 4.4331 - accuracy: 0.0579
Epoch 3/100
9/9 [==============================] - 0s 24ms/step - loss: 4.2570 - accuracy: 0.0386
Epoch 4/100
9/9 [==============================] - 0s 27ms/step - loss: 4.1615 - accuracy: 0.0695
Epoch 5/100
9/9 [==============================] - 0s 26ms/step - loss: 4.1458 - accuracy: 0.0618
Epoch 6/100
9/9 [==============================] - 0s 25ms/step - loss: 4.1257 - accuracy: 0.0618
Epoch 7/100
9/9 [==============================] - 0s 27ms/step - loss: 4.1010 - accuracy: 0.0618
Epoch 8/100
9/9 [==============================] - 0s 27ms/step - loss: 4.0706 - accuracy: 0.0656
Epoch 9/100
9/9 [==============================] - 0s 25ms/step - loss: 4.0184 - accuracy: 0.1042
Epoch 10/100
9/9 [==============================] - 0s 25ms/step - loss: 3.9422 - accuracy: 0.1467
Epoch 11/100
9/9 [=

This generate_code function takes in a seed text (initial text), a number of next_words to generate, the trained model, and max_sequence_len as inputs. The function generates new code based on the seed text using the trained model.

In [ ]:
def generate_code(seed_text, next_words, model, max_sequence_len):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = tf.keras.preprocessing.sequence.pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted_probs = model.predict(token_list)[0]
        predicted = np.argmax(predicted_probs)
        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " " + output_word
    return seed_text


The provided code generates Python code using the trained model. It initializes the generation process with a seed text and generates additional words based on the model's predictions.

In [ ]:
# Generate code using the trained model
generated_code = generate_code("add", 10, model, max_sequence_len)
print("Generated Python code:")
print(generated_code)

1/1 [==============================] - 0s 17ms/step
Generated Python code:
add a b return a b return result a n result
